In [19]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [20]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,  0.5, 0.5), (0.5, 0.5, 0.5))
])

In [21]:
train_data = torchvision.datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [22]:
image, label = train_data[0]

In [23]:
image.shape

torch.Size([3, 32, 32])

In [24]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [25]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5) #(12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2) #(12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) #(24, 10, 10) -> (24, 5, 5) -> Flatten (24 * 5 * 5)
        self.fc1  = nn.Linear(24 * 5 * 5, 120)
        self.fc2  = nn.Linear(120, 84)
        self.fc3  = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [26]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [27]:
for epoch in range(50):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0...
Loss: 2.1915
Training epoch 1...
Loss: 1.8338
Training epoch 2...
Loss: 1.5501
Training epoch 3...
Loss: 1.4243
Training epoch 4...
Loss: 1.3342
Training epoch 5...
Loss: 1.2546
Training epoch 6...
Loss: 1.1921
Training epoch 7...
Loss: 1.1314
Training epoch 8...
Loss: 1.0802
Training epoch 9...
Loss: 1.0357
Training epoch 10...
Loss: 0.9929
Training epoch 11...
Loss: 0.9541
Training epoch 12...
Loss: 0.9179
Training epoch 13...
Loss: 0.8849
Training epoch 14...
Loss: 0.8516
Training epoch 15...
Loss: 0.8239
Training epoch 16...
Loss: 0.7926
Training epoch 17...
Loss: 0.7677
Training epoch 18...
Loss: 0.7354
Training epoch 19...
Loss: 0.7140
Training epoch 20...
Loss: 0.6910
Training epoch 21...
Loss: 0.6690
Training epoch 22...
Loss: 0.6465
Training epoch 23...
Loss: 0.6207
Training epoch 24...
Loss: 0.6006
Training epoch 25...
Loss: 0.5765
Training epoch 26...
Loss: 0.5553
Training epoch 27...
Loss: 0.5325
Training epoch 28...
Loss: 0.5176
Training epoch 29...
Los

In [28]:
torch.save(net.state_dict(), 'trained_net.pth')

In [29]:
net  = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

C:\Users\Jose\AppData\Local\Temp\ipykernel_9260\1080633788.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [30]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

Accuracy: 65.13%


In [66]:
from os import listdir
from os.path import isfile, join

new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(f'images/{image_path}').convert('RGB')
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

mypath = "images"
image_paths = [f for f in listdir(mypath) if isfile(join(mypath, f))]
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image, title in zip(images, image_paths):
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]} - File name: {title}')

Prediction: plane - File name: aviao1.jpeg
Prediction: cat - File name: aviao10.jpg
Prediction: plane - File name: aviao2.jpg
Prediction: plane - File name: aviao3.jpg
Prediction: ship - File name: aviao4.jpg
Prediction: bird - File name: aviao5.jpg
Prediction: deer - File name: aviao6.jpg
Prediction: bird - File name: aviao7.jpg
Prediction: plane - File name: aviao8.jpg
Prediction: plane - File name: aviao9.jpg
Prediction: car - File name: carro1.jpeg
Prediction: plane - File name: carro10.jpeg
Prediction: car - File name: carro2.jpg
Prediction: car - File name: carro3.jpg
Prediction: car - File name: carro4.jpg
Prediction: car - File name: carro5.jpg
Prediction: truck - File name: carro6.jpg
Prediction: car - File name: carro7.jpg
Prediction: car - File name: carro8.jpg
Prediction: car - File name: carro9.jpg
Prediction: deer - File name: deer1.jpg
Prediction: deer - File name: deer10.jpg
Prediction: deer - File name: deer2.jpg
Prediction: deer - File name: deer3.jpg
Prediction: cat 

d:\python\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
